In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

from pprint import pprint
import math

from scipy.stats import kurtosis, skew

from IPython import embed
from IPython.terminal.embed import InteractiveShellEmbed

from sklearn.model_selection import KFold

import random
import seaborn as sns
import matplotlib.pyplot as plt
import shap
plt.rcParams['figure.figsize'] = (12,6)

import os
from os.path import join as pjoin

data_root = '../input/build-my-data'
print(os.listdir(data_root))

# Any results you write to the current directory are saved as output.

In [ ]:
def load_data(data='train',n=2):
    df = pd.DataFrame()
    for i in range(n) :
        if data=='train':
            if i > 8 :
                break
            dfpart = pd.read_pickle(pjoin(data_root,f'train_{i}.pkl'))
        elif data=='test':
            if i > 2 :
                break
            dfpart = pd.read_pickle(pjoin(data_root,f'test_{i}.pkl'))
        df = pd.concat([df,dfpart])
        del dfpart
    return df

In [ ]:
%%time
df_train = load_data(n=9)
df_test = load_data('test',n=4)

In [ ]:
#df_train.isnull().sum()
#df_test.isnull().sum()
df_all = pd.concat([df_train,df_test]).reset_index(drop=True)
print({"all":df_all.shape,
       "df_train":df_train.shape,
       "df_test":df_test.shape,})

In [ ]:
def generate_label(label,id_dfx):
    col_label=['fullVisitorId','totals_transactionRevenue']
    #Select only the id is in df_train for the label
    label = label[label.fullVisitorId.isin(id_dfx)].copy()
    label=label.reset_index(drop=True)
    #drop all columuns else fullvisitorsid and totaltransations
    for c in label.columns:
        if(c not in col_label ):
            label.drop(c,axis=1,inplace=True)
    #Select the id in train not in label       
    id_label = label.fullVisitorId.drop_duplicates()
    not_in_label=list(set(id_dfx) - set(id_label))
    zeros=[0 for c in range(0,len(not_in_label))]
    df_label_0=pd.DataFrame(list(zip(not_in_label, zeros)) ,columns=['fullVisitorId','totals_transactionRevenue'])
    #Contatane te two dataframe
    label=pd.concat([label,df_label_0]).reset_index(drop=True)
    return label

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
def three_month_after(train_begin,train_end,label_begin,label_end,verbose=False):
    tb=datetime.strptime(train_begin,'%Y-%m-%d')+relativedelta(months=3);
    te=datetime.strptime(train_end,'%Y-%m-%d')+relativedelta(months=3);
    lb=datetime.strptime(label_begin,'%Y-%m-%d')+relativedelta(months=3);
    le=datetime.strptime(label_end,'%Y-%m-%d')+relativedelta(months=3);
    if verbose:
        print("train---------------------------------")
        print( 'intial:',datetime.strptime(train_begin,'%Y-%m-%d'))
        print( 'After 3 Month:', tb.strftime('%Y-%m-%d'))
        print( 'intial:',datetime.strptime(train_end,'%Y-%m-%d'))
        print( 'After 3 Month:', te.strftime('%Y-%m-%d'))
        print("label---------------------------------")
        print( 'intial:',datetime.strptime(label_begin,'%Y-%m-%d'))
        print( 'After 3 Month:', lb.strftime('%Y-%m-%d'))
        print( 'intial:',datetime.strptime(label_end,'%Y-%m-%d'))
        print( 'After 3 Month:', le.strftime('%Y-%m-%d'))
    df_x = df_all[ (df_all.date >= tb.strftime('%Y-%m-%d')) & (df_all.date <= te.strftime('%Y-%m-%d'))].copy() #5,5 months(oct ---> march *0.5)
    df_x=df_x.reset_index(drop=True)
    #<-> 1.5 month 
    label = df_all[(df_all.date >= lb.strftime('%Y-%m-%d')) & (df_all.date <= le.strftime('%Y-%m-%d'))].copy() #2 months (may--->june)
    id_train = df_x.fullVisitorId.drop_duplicates()

    #Generate label
    label=generate_label(label,id_train).copy()
    
    return df_x,label

In [ ]:
# train_v2.csv - from August 1st 2016 to April 30th 2018.
# test_v2.csv - from May 1st 2018 to October 15th 2018.
# sample_submission_v2.csv - from December 1st 2018 to January 31st 2019


#test
df_test_x = df_all[(df_all.date >= "2018-05-01") & (df_all.date <= "2018-10-15")].copy()
df_test_x =df_test_x.reset_index(drop=True)
#df_test_y= "sample submission"

#fold1
#----------------------------train
col=['fullVisitorId','totals_transactionRevenue']
#train1 
#df_train1_x_agg.shape (377186, 67)
df_train1_x = df_all[df_all.date <= "2016-12-30"].copy() #5 months(august ---> dec)
df_train1_x=df_train1_x.reset_index(drop=True)
#<-> 1.5 month (jan , fev *0.5)
label_1 = df_all[(df_all.date >= "2017-02-15") & (df_all.date <= "2017-04-15")].copy() #2 months (feb*0.5--->april*0.5)
id_train1 = df_train1_x.fullVisitorId.drop_duplicates()

#Generate label
label_1=generate_label(label_1,id_train1).copy()

#items = set(not_in_label)
#set(new_list).issubset(items)

#len(id_train1)
#len(label_1.fullVisitorId.drop_duplicates())

df_train1_x.shape

In [ ]:
df_train2_x,label_2=three_month_after("2016-08-01","2016-12-30","2017-02-15","2017-04-15",verbose=True)

In [ ]:
#train3 
df_train3_x,label_3=three_month_after("2016-11-01","2017-03-30","2017-05-15","2017-07-15",verbose=True)

In [ ]:
df_train4_x,label_4=three_month_after("2017-02-01","2017-06-30","2017-08-15","2017-10-15",verbose=True)

In [ ]:
#train5 
df_train5_x,label_5=three_month_after("2017-05-01","2017-09-30","2017-11-15","2018-01-15",verbose=True)

In [ ]:
#train6
df_train6_x,label_6=three_month_after("2017-08-01","2017-12-30","2018-02-15","2018-04-15",verbose=True)

In [ ]:
from pandas.io.json import json_normalize
def json_it(df,col):
    for column in col:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        #df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        df = df.merge(column_as_df, right_index=True, left_index=True).copy()
    #print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    print("Done json it.... ")
    return df

In [ ]:
def group_by_transactionRevenue(df_x,y):
    cat_cols=["channelGrouping","device_browser","device_deviceCategory","device_operatingSystem","geoNetwork_city",
          "geoNetwork_continent","geoNetwork_country","geoNetwork_metro", "geoNetwork_networkDomain",
          "geoNetwork_region","geoNetwork_subContinent", "trafficSource_adContent",
          "trafficSource_adwordsClickInfo.adNetworkType","trafficSource_adwordsClickInfo.gclId",
          "trafficSource_adwordsClickInfo.slot","trafficSource_campaign","trafficSource_keyword",
          "trafficSource_medium","trafficSource_referralPath","trafficSource_source","customDimensions_value",
          "browser_category","browser_operatingSystem"]
    
    df_x_agg=df_x.groupby("fullVisitorId",as_index=False)['totals_transactionRevenue'].sum().sort_values("fullVisitorId").reset_index(drop=True).copy()
    y_agg = y.groupby("fullVisitorId",as_index=False)['totals_transactionRevenue'].sum().sort_values("fullVisitorId").reset_index(drop=True).copy()
    
    # log totals_transactionRevenue
    df_x_agg['totals_transactionRevenue'] = np.log1p(df_x_agg['totals_transactionRevenue'])
    y_agg['totals_transactionRevenue'] = np.log1p(y_agg['totals_transactionRevenue'])
    print("Done ")
    #df_x_agg = pd.merge(df_x_agg_sum,df_x_agg_last, how='left',on="fullVisitorId").sort_values("fullVisitorId").reset_index(drop=True).copy()
    return y_agg

In [ ]:
def timeSeries_2(df,col,agg):
    print("Selected column: ",col)
    if (agg=="sum"):
        print("Selected agg: ",agg)
        temp =df.groupby(["fullVisitorId", "Date_Month"],as_index=False)[col].sum().sort_values(["fullVisitorId","Date_Month"]).reset_index(drop=True).copy()
    elif (agg=="count"):
        print("Selected agg: ",agg)
        temp =df.groupby(["fullVisitorId", "Date_Month"],as_index=False)[col].count().sort_values(["fullVisitorId","Date_Month"]).reset_index(drop=True).copy()
    temp.reset_index(drop=True)
    
    if(col=='totals_transactionRevenue' and agg != "count"):
        print("log applied....... ")
        temp[col] = np.log1p(temp[col])
        
    print("Data shape : ",temp.shape)
    #temp['totals_transactionRevenue'] = np.log1p(temp['totals_transactionRevenue'])
    id_unique_df=list(df.fullVisitorId.drop_duplicates())
    print("numbers of uniques fullVisitorId: ",len(id_unique_df))
    months=list(df.Date_Month.drop_duplicates())
    months.sort()
    print("months  in df: ",months)
    
    nrows=temp.shape[0]
    previous="??"
    dict_time_series={}
    count=0
    visitors_col=[]
    agg_col=[]
    for m in months:
        dict_time_series[str(m)]=0.0
    for index, row in temp.iterrows():
        if previous != str(row["fullVisitorId"]) and previous!="??":
            #print(previous," ---now id :-- ",row["fullVisitorId"],"   ",dict_time_series)
            visitors_col.append(previous)
            agg_col.append(dict_time_series.copy())
            count=count+1
            for m in months:
                dict_time_series[str(m)]=0.0
            previous=str(row["fullVisitorId"])
            dict_time_series[str(row["Date_Month"])] = row[col]
            if index == nrows-1:
                print("the last: ")
                print(previous," ----- ",dict_time_series)
                visitors_col.append(previous)
                agg_col.append(dict_time_series.copy())
                count=count+1
            
        else:
            #for i in range(visitor_agg.shape[0]):
            #print(visitor_agg.iloc[i]['Date_Month'],visitor_agg.iloc[i][col])
            dict_time_series[str(row["Date_Month"])] = row[col]
            previous=str(row["fullVisitorId"])
            if index == nrows-1:
                print("the last: ")
                print(previous," ----- ",dict_time_series)
                visitors_col.append(previous)
                agg_col.append(dict_time_series.copy())
                count=count+1
    print("index: ",index,"counter",count)
    new_col_name=str(col)+"_timeSeries_"+str(agg)
    d = {"fullVisitorId":visitors_col,new_col_name:agg_col}
    df = pd.DataFrame(d)
   #
    df_j=json_it(df.copy(),[new_col_name])
    
    #
    cols_timeS=[]
    for m in months:
        cols_timeS.append(new_col_name+'.'+str(m))
    print(cols_timeS)
    df_values=df_j[cols_timeS].values
    print('get values done....')
    return df_values

In [ ]:
print(" -------------------------1---------------------------- ")
#train1
timeS_trans_sum1=timeSeries_2(df_train1_x,'totals_transactionRevenue',"sum").copy()
timeS_trans_count1=timeSeries_2(df_train1_x,'totals_transactionRevenue',"count").copy()
timeS_bounce_sum1=timeSeries_2(df_train1_x,'totals_bounces',"sum").copy()
timeS_hit_sum1=timeSeries_2(df_train1_x,'totals_hits',"sum").copy()
timeS_pageV_sum1=timeSeries_2(df_train1_x,'totals_pageviews',"sum").copy()
print(" --------------------------2---------------------------- ")
#train2
timeS_trans_sum2=timeSeries_2(df_train2_x,'totals_transactionRevenue',"sum").copy()
timeS_trans_count2=timeSeries_2(df_train2_x,'totals_transactionRevenue',"count").copy()
timeS_bounce_sum2=timeSeries_2(df_train2_x,'totals_bounces',"sum").copy()
timeS_hit_sum2=timeSeries_2(df_train2_x,'totals_hits',"sum").copy()
timeS_pageV_sum2=timeSeries_2(df_train2_x,'totals_pageviews',"sum").copy()
print(" ---------------------------3--------------------------- ")
#train3
timeS_trans_sum3=timeSeries_2(df_train3_x,'totals_transactionRevenue',"sum").copy()
timeS_trans_count3=timeSeries_2(df_train3_x,'totals_transactionRevenue',"count").copy()
timeS_bounce_sum3=timeSeries_2(df_train3_x,'totals_bounces',"sum").copy()
timeS_hit_sum3=timeSeries_2(df_train3_x,'totals_hits',"sum").copy()
timeS_pageV_sum3=timeSeries_2(df_train3_x,'totals_pageviews',"sum").copy()
print(" ----------------------------4--------------------------- ")
#train4
timeS_trans_sum4=timeSeries_2(df_train4_x,'totals_transactionRevenue',"sum").copy()
timeS_trans_count4=timeSeries_2(df_train4_x,'totals_transactionRevenue',"count").copy()
timeS_bounce_sum4=timeSeries_2(df_train4_x,'totals_bounces',"sum").copy()
timeS_hit_sum4=timeSeries_2(df_train4_x,'totals_hits',"sum").copy()
timeS_pageV_sum4=timeSeries_2(df_train4_x,'totals_pageviews',"sum").copy()
print(" -----------------------------5-------------------------- ")
#train5
timeS_trans_sum5=timeSeries_2(df_train5_x,'totals_transactionRevenue',"sum").copy()
timeS_trans_count5=timeSeries_2(df_train5_x,'totals_transactionRevenue',"count").copy()
timeS_bounce_sum5=timeSeries_2(df_train5_x,'totals_bounces',"sum").copy()
timeS_hit_sum5=timeSeries_2(df_train5_x,'totals_hits',"sum").copy()
timeS_pageV_sum5=timeSeries_2(df_train5_x,'totals_pageviews',"sum").copy()
print(" -----------------------------6-------------------------- ")
#train6
timeS_trans_sum6=timeSeries_2(df_train6_x,'totals_transactionRevenue',"sum").copy()
timeS_trans_count6=timeSeries_2(df_train6_x,'totals_transactionRevenue',"count").copy()
timeS_bounce_sum6=timeSeries_2(df_train6_x,'totals_bounces',"sum").copy()
timeS_hit_sum6=timeSeries_2(df_train6_x,'totals_hits',"sum").copy()
timeS_pageV_sum6=timeSeries_2(df_train6_x,'totals_pageviews',"sum").copy()

In [ ]:
# group by labels 2 months
label_1=group_by_transactionRevenue(df_train1_x,label_1)
label_2=group_by_transactionRevenue(df_train2_x,label_2)
label_3=group_by_transactionRevenue(df_train3_x,label_3)
label_4=group_by_transactionRevenue(df_train4_x,label_4)
label_5=group_by_transactionRevenue(df_train5_x,label_5)
label_6=group_by_transactionRevenue(df_train6_x,label_6)

In [ ]:
def rechape_train(list_timeS):
    nb_features=5
    first=list_timeS[0]
    X_train_r = np.zeros((len(first), nb_features, len(list_timeS)))
    i=0
    for tS in list_timeS:
        X_train_r[:, :,i ] = tS[:, :nb_features]
        i=i+1
    print("reshape done......")
    return X_train_r

In [ ]:
# creating the rechape data x
#train1
X_train1_r=rechape_train([timeS_trans_sum1,timeS_trans_count1,timeS_bounce_sum1,timeS_hit_sum1,timeS_pageV_sum1])
#train2
X_train2_r=rechape_train([timeS_trans_sum2,timeS_trans_count2,timeS_bounce_sum2,timeS_hit_sum2,timeS_pageV_sum2])
#train3
X_train3_r=rechape_train([timeS_trans_sum3,timeS_trans_count3,timeS_bounce_sum3,timeS_hit_sum3,timeS_pageV_sum3])
#train4
X_train4_r=rechape_train([timeS_trans_sum4,timeS_trans_count4,timeS_bounce_sum4,timeS_hit_sum4,timeS_pageV_sum4])
#train5
X_train5_r=rechape_train([timeS_trans_sum5,timeS_trans_count5,timeS_bounce_sum5,timeS_hit_sum5,timeS_pageV_sum5])
#train6
X_train6_r=rechape_train([timeS_trans_sum6,timeS_trans_count6,timeS_bounce_sum6,timeS_hit_sum6,timeS_pageV_sum6])

In [ ]:
from keras import backend
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true)))

from keras.layers import Activation, Input, Dense, Embedding, concatenate, Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Conv1D,Input,Dense,MaxPooling1D,Dropout,BatchNormalization,Activation

In [ ]:
def build_model(filter=32):
    """""
    input_layer_broswer = Input( shape=(1,) , name='broswer')
    emb1 = Embedding( input_dim = 1+np.max( dc['broswer'] ), output_dim=3 )(input_layer_broswer)
    emb1 = Flatten( )(emb1)
    
    input_layer_deviceCategory = Input( shape=(1,) , name='deviceCategory')
    emb2 = Embedding( input_dim = 1+np.max( dc['deviceCategory'] ), output_dim=3 )(input_layer_deviceCategory)
    emb2 = Flatten( )(emb2)
    
    """""
    #block 1
    input_layer_num = Input(shape=(5,5),name='timeS_sum')
    conv1=Conv1D(filter,kernel_size=2,name='conv1',padding="same")(input_layer_num)
    bn1=BatchNormalization(epsilon=0.01,name='bn1')(conv1)
    bn1=Activation('relu',name='bn_activation1')(bn1)
    conv1_2=Conv1D(filter,kernel_size=2,name='conv1_2',padding="same")(bn1)
    act=Activation('relu',name='activation')(conv1_2)

    #block2
    conv2=Conv1D(filter*2,kernel_size=2,name='conv2',padding="same")(act)
    bn2=BatchNormalization(epsilon=0.01,name='bn2')(conv2)
    bn2=Activation('relu',name='bn_activation2')(bn2)
    conv2_1=Conv1D(filter*2,kernel_size=2,name='conv2_1',padding="same")(bn2)
    drop=Dropout(rate=0.3,name='drop')(conv2_1)
    
    drop = Flatten( )(drop)
    dense1 = Dense(units=1000, name='dense1')(drop)
    dense1 = Activation('relu', name='activation1')(dense1)
    
    output_layer = Dense(units=1)(dense1)
    output_layer=Activation('sigmoid',name='activation2')(output_layer)
    
    model = Model([input_layer_num], output_layer)
    
    opt = SGD()
    model.compile(optimizer=opt, loss=rmse, metrics=['accuracy'])
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
valid_rmse={}

In [ ]:
#fold1
fold1=model.fit(x=X_train1_r, y=label_1["totals_transactionRevenue"], batch_size=100, epochs=4, validation_data=[X_train2_r,label_2["totals_transactionRevenue"]])
valid_rmse.update({'fold1': min(fold1.history["val_loss"])}) 

In [ ]:
#fold2
model = build_model()
fold2=model.fit(x=X_train2_r, y=label_2["totals_transactionRevenue"], batch_size=100, epochs=4, validation_data=[X_train3_r,label_3["totals_transactionRevenue"]])
valid_rmse.update({'fold2': min(fold2.history["val_loss"])}) 

In [ ]:
#fold3
model = build_model()
fold3=model.fit(x=X_train3_r, y=label_3["totals_transactionRevenue"], batch_size=100, epochs=4, validation_data=[X_train4_r,label_4["totals_transactionRevenue"]])
valid_rmse.update({'fold3': min(fold3.history["val_loss"])}) 

In [ ]:
#fold4
model = build_model()
fold4=model.fit(x=X_train4_r, y=label_4["totals_transactionRevenue"], batch_size=100, epochs=4, validation_data=[X_train5_r,label_5["totals_transactionRevenue"]])
valid_rmse.update({'fold4': min(fold4.history["val_loss"])}) 

In [ ]:
#fold5
model = build_model()
fold5=model.fit(x=X_train5_r, y=label_5["totals_transactionRevenue"], batch_size=100, epochs=4, validation_data=[X_train6_r,label_6["totals_transactionRevenue"]])
valid_rmse.update({'fold5': min(fold5.history["val_loss"])}) 

In [ ]:
#plot k fold logloss
plt.plot(range(len(valid_rmse)), valid_rmse.values())
plt.xticks(range(len(valid_rmse)), list(valid_rmse.keys()))
plt.xticks(rotation=90)

plt.title('model logloss')  
plt.ylabel('logloss')  
plt.xlabel('fold')  
plt.legend(['valid']) 
plt.show()

In [ ]:
#train2
df_train2_x,label_2=three_month_after("2016-08-01","2017-01-15","2017-03-01","2017-04-30",verbose=True)

In [ ]:
# reshape train data
#nb_features=5
#X_train1_r = np.zeros((len(timeS_trans_sum1), nb_features, 2))
#X_train1_r[:, :, 0] = timeS_trans_sum1[:, :nb_features]
#X_train1_r[:, :, 1] = timeS_trans_count1[:, :nb_features]

#X_train2_r = np.zeros((len(timeS_trans_sum2), nb_features, 2))
#X_train2_r[:, :, 0] = timeS_trans_sum2[:, :nb_features]
#X_train2_r[:, :, 1] = timeS_trans_count2[:, :nb_features]

In [ ]:
def timeSeries(df,col,agg):
    print("Selected column: ",col)
    if (agg=="sum"):
        print("Selected agg: ",agg)
        temp =df.groupby(["fullVisitorId", "Date_Month"],as_index=False)[col].sum().sort_values(["fullVisitorId","Date_Month"]).reset_index(drop=True).copy()
    elif (agg=="count"):
        print("Selected agg: ",agg)
        temp =df.groupby(["fullVisitorId", "Date_Month"],as_index=False)[col].count().sort_values(["fullVisitorId","Date_Month"]).reset_index(drop=True).copy()
    temp.reset_index(drop=True)
    #temp['totals_transactionRevenue'] = np.log1p(temp['totals_transactionRevenue'])
    if(col=='totals_transactionRevenue' and agg != "count"):
        print("log applied....... ")
        temp[col] = np.log1p(temp[col])
        
    print("Data shape : ",temp.shape)
    id_unique_df=list(df.fullVisitorId.drop_duplicates())
    print("numbers of uniques fullVisitorId: ",len(id_unique_df))
    months=list(df.Date_Month.drop_duplicates())
    months.sort()
    print("months  in df: ",months)
    visitors_col=[]
    agg_col=[]
    for id_ in id_unique_df:
        visitor_agg=temp[temp["fullVisitorId"]== id_]
        visitor_agg.reset_index(drop=True)
        dict_time_series={}
        for m in months:
            dict_time_series[str(m)]=0.0
        for i in range(visitor_agg.shape[0]):
            #print(visitor_agg.iloc[i]['Date_Month'],visitor_agg.iloc[i][col])
            dict_time_series[str(visitor_agg.iloc[i]['Date_Month'])] = visitor_agg.iloc[i][col]
        #print("id: ",id_," time series : ",dict_time_series)
        visitors_col.append(id_)
        agg_col.append(dict_time_series)
    d = {"fullVisitorId":visitors_col,str(col)+"_timeSeries_"+str(agg):agg_col}
    df = pd.DataFrame(d)
    print("done.....")
    return df
#timeSeries(df_train1_x,'totals_transactionRevenue')"""